# Treinamento

## Inicialização

In [ ]:
# Config Inicial
import json
import mlflow.pyfunc
import shutil
import pandas as pd
import requests
from sklearn.metrics import r2_score
import mlflow
from mlflow.tracking import MlflowClient

import os
import signal
import subprocess


### Leitura

In [ ]:
# Descomentar, quando quiser carregar para o treinamento, dados originais defasados para primeira execução do treinamento
#shutil.copyfile("dataset/brasil_estado_cidade.csv", "dataset/dados_treinamento.csv")

# Carregar o conjunto de dados
df_treinamento = pd.read_csv("dataset/dados_treinamento.csv", encoding="utf-8")

# Eliminando registros com valores null
df_treinamento.dropna(inplace=True)

# float64
df_treinamento = df_treinamento.astype({col: 'float64' for col in df_treinamento.select_dtypes(include='int').columns})

df_treinamento.head()

In [ ]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")

mlflow.set_experiment("ecd15")

### Amostragem

In [ ]:
# amostra
df_sample = df_treinamento.sample(n=20, random_state=42)

x_features = df_sample.drop(["price_brl"], axis=1)  # Features
y_target = df_sample["price_brl"]  # Variável alvo

df_sample.head()

In [ ]:
# Gerando o JSON no formato esperado pelo MLflow Serve
dados = {"instances": x_features.to_dict(orient="records")}
json_data = json.dumps(dados)

In [ ]:
previsao = requests.post(
    "http://localhost:5000/invocations",
    headers={"Content-Type": "application/json"},
    data=json_data
)
print("HTTP STATUS", previsao.status_code)

In [ ]:
previsao.text

### Carregar e identificar o melhor modelo treinado

In [ ]:

def carregar_modelo_com_melhor_r2(experimento_nome):
    client = MlflowClient()

    # Pega o experimento
    experimento = client.get_experiment_by_name(experimento_nome)
    if not experimento:
        raise ValueError(f"Experimento '{experimento_nome}' não encontrado.")
    
    experiment_id = experimento.experiment_id

    melhor_modelo = {
        "r2": float('-inf'),
        "model_uri": None,
        "nome_modelo": None,
        "versao_modelo": None,
        "run_id": None
    }

    modelo_api = {
        "r2": float('-inf'),
        "model_uri": None,
        "nome_modelo": None,
        "versao_modelo": None,
        "run_id": None
    }

    # Pega todas as runs ordenadas por R² (decrescente)
    runs = client.search_runs(
        experiment_ids=[experiment_id],
        order_by=["metrics.r2 DESC"]
    )

    modelos_registrados = client.search_registered_models()

    

    for modelo in modelos_registrados:
        
        nome_modelo = modelo.name

        for versao in modelo.latest_versions:
            # Busca informações completas da versão
            versoes = client.search_model_versions(f"name='{nome_modelo}'")
            for v in versoes:
                run = client.get_run(v.run_id)

                # Verifica se pertence ao experimento desejado
                if run.info.experiment_id != experiment_id:
                    continue

                r2 = run.data.metrics.get("r2")

                source_path = v.source
                artifact_path = source_path.split("/")[-1]
                model_uri = f"runs:/{v.run_id}/{artifact_path}"

                print(f"Modelo '{modelo.name}' versão {v.version} da run {v.run_id} com R² = {r2}")

                if v.current_stage == "Production":
                    modelo_api.update({
                        "r2": r2,
                        "model_uri": model_uri,
                        "nome_modelo": nome_modelo,
                        "versao_modelo": v.version,
                        "run_id": v.run_id
                    })
                
                if r2 is not None and r2 > melhor_modelo["r2"] and v.current_stage != "Archived":
                    melhor_modelo.update({
                        "r2": r2,
                        "model_uri": model_uri,
                        "nome_modelo": nome_modelo,
                        "versao_modelo": v.version,
                        "run_id": v.run_id
                    })
    
    
    print(f"Melhor modelo '{melhor_modelo['nome_modelo']}' versão {melhor_modelo['versao_modelo']} da run {melhor_modelo['run_id']} com R² = {melhor_modelo['r2']:.4f}")

    print(f"Modelo API '{modelo_api['nome_modelo']}' versão {modelo_api['versao_modelo']} da run {modelo_api['run_id']} com R² = {modelo_api['r2']:.4f}")

    modelo_carregado = mlflow.pyfunc.load_model(melhor_modelo["model_uri"])
    
    return modelo_carregado, melhor_modelo, modelo_api
    

# Carregando modelo com melhor R2 
loaded_model, melhor_modelo, modelo_api = carregar_modelo_com_melhor_r2("ecd15")

if not melhor_modelo["model_uri"]:
    print("Nenhum modelo em 'Staging' encontrado para esse experimento.")

existe_modelo_api = True
if not modelo_api["model_uri"]:
    existe_modelo_api = False
    print("Nenhum modelo em 'Production' encontrado para esse experimento.")

melhor_modelo, modelo_api


## Predição de melhor modelo carregado x modelo API

In [ ]:
# Fazer a previsão
predictions = loaded_model.predict(x_features)
# Adicionar as previsões ao DataFrame
    
df_sample["predicao_melhor_modelo"] = predictions

# converter previsões para o mesmo tipo da variável alvo
df_sample["predicao_melhor_modelo"] = df_sample["predicao_melhor_modelo"].astype(float).round(2)

colunas_exibidas = ["price_brl", "predicao_melhor_modelo"]

if existe_modelo_api:
    previsao_json = json.loads(previsao.text)

    # Extrair as previsões do JSON e adicioná-las ao DataFrame
    df_sample["predicao_modelo_api"] = [round(pred, 2) for pred in previsao_json["predictions"]]

    colunas_exibidas = ["price_brl", "predicao_modelo_api", "predicao_melhor_modelo"]

df_sample[colunas_exibidas]

### Comparação R² Score dos modelos

In [ ]:
# Calcular R²
r2_melhor_modelo = r2_score(df_sample["price_brl"], df_sample["predicao_melhor_modelo"])

if existe_modelo_api:
    r2_api = r2_score(df_sample["price_brl"], df_sample["predicao_melhor_modelo"])
    print(f"R² API: {r2_melhor_modelo:.4f} R² Melhor Modelo: {r2_api:.4f}")
else:
    print(f"R² Melhor Modelo: {r2_melhor_modelo:.4f}")

### Promover melhor modelo para Produção

In [ ]:
def parar_modelo_mlflow1():
    try:
        resultado = subprocess.check_output("lsof -i :5000 -sTCP:LISTEN -nP", shell=True)
        linhas = resultado.decode().strip().split("\n")[1:]  # ignora cabeçalho
        for linha in linhas:
            if "mlflow" in linha.lower():
                pid = int(linha.split()[1])
                os.kill(pid, signal.SIGTERM)
                print(f"Servidor MLflow (PID {pid}) parado com sucesso.")
            else:
                print("⚠️ Porta 5000 ocupada por outro processo que não é o MLflow. Ignorado.")
    except subprocess.CalledProcessError:
        print("Nenhum processo ouvindo na porta 5000 foi encontrado.")

def promover_modelo_para_producao(modelo):
    client = MlflowClient()

    nome_modelo = modelo["nome_modelo"]
    versao_modelo = modelo["versao_modelo"]

    # Agora vamos arquivar os que ainda estão em Staging
    todos_modelos = client.search_model_versions("name LIKE '%_model'")
    modelos_ativos = [v for v in todos_modelos if v.current_stage != "Archived"]

    
    for v in modelos_ativos:
        client.transition_model_version_stage(
             name=v.name,
             version=v.version,
             stage="Archived"
        )
        print(f"Modelo { v.name } Versão {v.version} arquivado.")

    # Promover o modelo selecionado
    client.transition_model_version_stage(
        name=nome_modelo,
        version=versao_modelo,
        stage="Production",
        archive_existing_versions=True  # arquiva automaticamente os outros em Production
    )
    print(f"Modelo '{nome_modelo}' versão {versao_modelo} promovido para 'Production'.")


def servir_melhor_modelo(experimento_nome, modelo):
    client = MlflowClient()
    
    # Parar modelo atual
    os.system("bash parar_mlflow_serve.sh")

    model_uri = modelo["model_uri"]

    promover_modelo_para_producao(modelo)

    #Tentativas de tentar parar o servidor e subir via notebook não funcionaram corretamente, deixando a utilização lenta.
    print("É necessário parar e inicializar o docker novamente para carregar novo modelo na API automaticamente")

    # Reinicializar api com novo modelo
    os.system(f"bash reiniciar_mlflow_serve.sh {model_uri}")

servir_melhor_modelo("ecd15", melhor_modelo)

Modelo gradient_boosting_model Versão 26 arquivado.


Modelo linear_regression_model Versão 26 arquivado.


Modelo decision_tree_model Versão 26 arquivado.


Modelo xgboost_model Versão 24 arquivado.


Modelo gradient_boosting_model Versão 24 arquivado.


...

Modelo decision_tree_model Versão 5 arquivado.
Modelo gradient_boosting_model Versão 4 arquivado.
Modelo linear_regression_model Versão 4 arquivado.


Modelo decision_tree_model Versão 4 arquivado.
Modelo gradient_boosting_model Versão 3 arquivado.
Modelo linear_regression_model Versão 3 arquivado.


Modelo decision_tree_model Versão 3 arquivado.
Modelo gradient_boosting_model Versão 2 arquivado.
Modelo linear_regression_model Versão 2 arquivado.


Modelo decision_tree_model Versão 2 arquivado.
Modelo gradient_boosting_model Versão 1 arquivado.
Modelo linear_regression_model Versão 1 arquivado.


Modelo decision_tree_model Versão 1 arquivado.
